In [1]:
# imports
import pdb
from tqdm import tqdm

from IPython.display import display

import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

from dask_ml.preprocessing import StandardScaler as dStandardScaler
from dask_ml.xgboost import XGBRegressor as dXGBRegressor
from dask_ml.model_selection import train_test_split as dtrain_test_split

In [2]:
# dask tasks progress bar
pbar = ProgressBar()
pbar.register()

# strategy

- what do we want to predict?
    - given last 6 days of data, what would a metric look like tomorrow?
    - given last 6 days of data, what would a metric look like n days from today?

- whats the baseline
    - tomo same as today
    - tomo avg of last 6 days

- features
    - vanilla
    - 6 day aggs

- what methods to explore
    - linear regression
    - ensembles - xgb, rf, ada
    - lstm

- how do we deal with the large amount of data
    - lets only focus on recall (prioritize predicting failing drives)
    - only apply methods on failed df, which can fit in mem easily

# Read Data

In [3]:
# read df and keep seagate data
df = dd.read_parquet('../data/interim/data_Q3_2020_parquet')
df = df[(df['model'].str.startswith('S')) | (df['model'].str.startswith('ZA'))]

In [4]:
# set which columns are metadata and which ones are smart attribuetes
meta_cols = ['date', 'serial_number', 'model', 'capacity_bytes', 'failure']
all_smart_cols = [c for c in df.columns if c.startswith('smart_')]

In [5]:
# get the last date for each working drive
maxdate = df['date'].max().compute()
working_lastdate_df = df[df['date']==maxdate].compute().set_index('date')
display(working_lastdate_df.head())

# get the last date for each failed drive
failed_lastdate_df = df[df['failure']==1].compute().set_index('date')
display(failed_lastdate_df.head())

[########################################] | 100% Completed | 56.4s
[########################################] | 100% Completed | 50.4s


,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
date,,,,,,,,,,,,,,,,,,,,,
2020-09-30,ZLW0EGC6,ST12000NM001G,1.200014e+13,0.0,78.0,64506312.0,NaN,NaN,99.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-30,Z305B2QN,ST4000DM000,4.000787e+12,0.0,117.0,124061384.0,NaN,NaN,91.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-30,ZJV0XJQ4,ST12000NM0007,1.200014e+13,0.0,76.0,40638464.0,NaN,NaN,89.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-30,ZJV0XJQ3,ST12000NM0007,1.200014e+13,0.0,83.0,212581936.0,NaN,NaN,98.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-30,ZA16NQJR,ST8000NM0055,8.001563e+12,0.0,78.0,60165320.0,NaN,NaN,89.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


[########################################] | 100% Completed | 54.0s


,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
date,,,,,,,,,,,,,,,,,,,,,
2020-07-01,ZCH0DG22,ST12000NM0007,1.200014e+13,1.0,83.0,212778168.0,NaN,NaN,99.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-01,ZLW0G6FJ,ST12000NM001G,1.200014e+13,1.0,78.0,70732912.0,NaN,NaN,99.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-02,ZHZ50G0W,ST12000NM0008,1.200014e+13,1.0,82.0,156403224.0,NaN,NaN,97.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-02,Z302SYQ3,ST4000DM000,4.000787e+12,1.0,117.0,135695912.0,NaN,NaN,91.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-02,ZA13YP8K,ST8000DM002,8.001563e+12,1.0,76.0,38002048.0,NaN,NaN,85.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# combine working and failed drives data to create lastdate_info df
lastdate_df = pd.concat(
    [
        working_lastdate_df,
        failed_lastdate_df,
    ],
    axis=0,
    join='outer',
)
assert lastdate_df.shape[0]==(working_lastdate_df.shape[0] + failed_lastdate_df.shape[0])
lastdate_df.head()

,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
date,,,,,,,,,,,,,,,,,,,,,
2020-09-30,ZLW0EGC6,ST12000NM001G,1.200014e+13,0.0,78.0,64506312.0,NaN,NaN,99.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-30,Z305B2QN,ST4000DM000,4.000787e+12,0.0,117.0,124061384.0,NaN,NaN,91.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-30,ZJV0XJQ4,ST12000NM0007,1.200014e+13,0.0,76.0,40638464.0,NaN,NaN,89.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-30,ZJV0XJQ3,ST12000NM0007,1.200014e+13,0.0,83.0,212581936.0,NaN,NaN,98.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-30,ZA16NQJR,ST8000NM0055,8.001563e+12,0.0,78.0,60165320.0,NaN,NaN,89.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# which serials belong to which class
working_serials = working_lastdate_df['serial_number'].unique()
failed_serials = failed_lastdate_df['serial_number'].unique()

# subset to work with coz mem constraints
working_serials_sample = np.random.choice(working_serials, size=int(0.01*len(working_serials)), replace=True)

In [8]:
# get timeseries df
working_ts_df = df[df['serial_number'].isin(working_serials_sample)].compute()
print(working_ts_df.shape)

failed_ts_df = df[df['serial_number'].isin(failed_serials)].compute()
print(failed_ts_df.shape)

[########################################] | 100% Completed | 52.7s
(90473, 131)
[########################################] | 100% Completed | 50.9s
(12242, 131)


# Correlations with Failure

In [9]:
# see how the values on the last day differ across healthy and failing drives
corr = lastdate_df[all_smart_cols + ['failure']].corr()
top_feats_corr = corr['failure'].abs().sort_values(ascending=False)
top_feats_corr.head(50)

failure                 1.000000
smart_187_normalized    0.348686
smart_5_raw             0.195944
smart_184_normalized    0.158263
smart_184_raw           0.158263
smart_197_raw           0.123470
smart_198_raw           0.123466
smart_223_normalized    0.110165
smart_11_normalized     0.110165
smart_188_normalized    0.109243
smart_11_raw            0.105004
smart_223_raw           0.105004
smart_225_raw           0.095077
smart_225_normalized    0.092230
smart_187_raw           0.068909
smart_188_raw           0.057044
smart_183_normalized    0.030141
smart_16_raw            0.026986
smart_17_raw            0.026983
smart_231_raw           0.026728
smart_177_normalized    0.026728
smart_232_raw           0.023958
smart_12_raw            0.019973
smart_7_normalized      0.014367
smart_183_raw           0.013316
smart_3_raw             0.013231
smart_173_raw           0.013174
smart_10_normalized     0.013142
smart_200_raw           0.012958
smart_3_normalized      0.012300
smart_233_

In [10]:
corr['smart_5_normalized'].abs().sort_values(ascending=False)

smart_196_raw           1.000000
smart_5_normalized      1.000000
smart_196_normalized    1.000000
smart_10_normalized     0.992726
smart_198_normalized    0.990144
                          ...   
smart_252_raw                NaN
smart_254_normalized         NaN
smart_254_raw                NaN
smart_255_normalized         NaN
smart_255_raw                NaN
Name: smart_5_normalized, Length: 127, dtype: float64

In [ ]:
# visualize feats
stats_to_plot = [7, 12, 223, 225, 183, 16, 17, 231, 232, 177]
cols_to_plot = [f'smart_{c}_raw' for c in stats_to_plot]
cols_to_plot += [f'smart_{c}_normalized' for c in stats_to_plot]

for col in cols_to_plot:
    fig, ax = plt.subplots(1, 2, figsize=(12, 8))
    for dev in tqdm(working_serials_sample):
        sns.lineplot(
            x=working_ts_df[working_ts_df['serial_number']==dev]['date'],
            y=working_ts_df[working_ts_df['serial_number']==dev][col],
            ax=ax[0],
        )
    for dev in tqdm(failed_serials):
        sns.lineplot(
            x=failed_ts_df[failed_ts_df['serial_number']==dev]['date'],
            y=failed_ts_df[failed_ts_df['serial_number']==dev][col],
            ax=ax[1],
        )
    plt.xticks(rotation=90)
    plt.title(col)
    plt.show()

interesting smart metrics where diff in working vs fail

### current run

- 5, 184, 187, 188, 197, 198,      , 196,  222
- 11, 223, 225                     , 22, 24, , 226, 231
- graph - 223,225,183,16,17,231,177,232,12,7

### previous run

- 2, 5, 12, 183, 184, 187, 188, 196, 197, 198, 222
- 11, 22, 24, 225, 226, 231

### previous run
- 187, 184, 197, 198, 196, 188, 5, 183, 1
- 11, 232, 177, 231, 235, 233, 173, 22, 

In [ ]:
# CRITICAL_STATS = [1, 5, 7, 10, 187, 188, 190, 193, 197, 198, 241]
CRITICAL_STATS = [5, 183, 184, 187, 188, 196, 197, 198, 222]
crit_cols_raw = ['smart_{}_raw'.format(i) for i in CRITICAL_STATS]
crit_cols_normalized = ['smart_{}_normalized'.format(i) for i in CRITICAL_STATS]
smart_cols = crit_cols_raw + crit_cols_normalized


Y_COL = 'smart_1_normalized'
X_COLS = [i for i in smart_cols if i.split('_')[1]!=Y_COL.split('_')[1]]

In [ ]:
final_keep_smart_stats = [5, 183, 184, 187, 188, 196, 197, 198, 222]
final_keep_cols = [f'smart_{i}_raw' for i in final_keep_smart_stats]
final_keep_cols += [f'smart_{i}_normalized' for i in final_keep_smart_stats]
final_keep_cols

In [ ]:
failed_ts_df = failed_ts_df[meta_cols + final_keep_cols]
working_ts_df = failed_ts_df[meta_cols + final_keep_cols]

# Preprocess

In [ ]:
# config
NDAYS_DATA = 6
NDAYS_TO_PREDICT = 1

In [ ]:
# scalers and models
input_scaler = StandardScaler()
target_scaler = StandardScaler()

adaboost_regr = AdaBoostRegressor()
randomforest_regr = RandomForestRegressor()

In [ ]:
X_COLS_SET = set(X_COLS)
def get_preds(curr_dev_df):
    pdb.set_trace()
    
    # not enough features
    if len(X_COLS.difference(curr_dev_df.columns)) != 0:
        return
    
    preds = [None]*len(curr_dev_df)
    for i in range(len(curr_dev_df) - NDAYS_DATA - NDAYS_TO_PREDICT):

        x = input_scaler.fit_transform(
            curr_dev_df[X_COLS + [Y_COL]].iloc[i: i+NDAYS_DATA+NDAYS_TO_PREDICT]
        )
        y = target_scaler.fit_transform(
            curr_dev_df[[Y_COL]].iloc[i: i+NDAYS_DATA+NDAYS_TO_PREDICT]
        )

        adaboost_regr.fit(
            x[:NDAYS_DATA], 
            y[:NDAYS_DATA].ravel(),
        )

        preds[i+NDAYS_DATA+NDAYS_TO_PREDICT-1] = target_scaler.inverse_transform(
            adaboost_regr.predict(y[np.newaxis, -1])
        )[0]
    return pd.Series(preds, index=curr_dev_df.index)

ret = failed_ts_df.groupby('serial_number').apply(get_preds)
ret.head()

In [ ]:
def get_preds(curr_dev_df):
#     pdb.set_trace()
#     curr_dev_df = curr_dev_df.compute()
    preds = [None]*len(curr_dev_df)
    for i in range(len(curr_dev_df) - NDAYS_DATA):

        x = input_scaler.fit_transform(curr_dev_df[X_COLS + [Y_COL]].iloc[i: i+NDAYS_DATA+1])
        y = target_scaler.fit_transform(curr_dev_df[[Y_COL]].iloc[i: i+NDAYS_DATA+1])

        adaboost_regr.fit(
            x[:NDAYS_DATA], 
            y[:NDAYS_DATA].ravel(),
        )

        preds[i+NDAYS_DATA] = target_scaler.inverse_transform(
            adaboost_regr.predict(data[np.newaxis, -1])
        )[0]
    return pd.Series(preds, index=curr_dev_df.index)

In [ ]:
preds_meta = df._meta.drop([c for c in df.columns if c != Y_COL], axis=1)

ret = df.groupby('serial_number').apply(get_preds, meta=preds_meta).compute()
ret.head()

In [ ]:
preds = [None]*len(device_df)
for i in range(len(device_df) - NDAYS_DATA):
    
    x = input_scaler.fit_transform(device_df[X_COLS + [Y_COL]].iloc[i: i+NDAYS_DATA+1])
    y = target_scaler.fit_transform(device_df[[Y_COL]].iloc[i: i+NDAYS_DATA+1])
    
    adaboost_regr.fit(
        x[:NDAYS_DATA], 
        y[:NDAYS_DATA].ravel(),
    )
    
    preds[i+NDAYS_DATA] = target_scaler.inverse_transform(
        adaboost_regr.predict(data[np.newaxis, -1])
    )[0]
    
preds = pd.Series(preds, index=device_df.index)
preds

In [ ]:
y_true = device_df[y_col]

In [ ]:
sklearn.metrics.mean_squared_error(preds.iloc[NDAYS_DATA+1:], y_true.iloc[NDAYS_DATA+1:])

In [ ]:
xtrain = device_df[x_cols].iloc[:train_idx]
xtest = device_df[x_cols].iloc[train_idx:]
ytrain = device_df[y_col].iloc[:train_idx]
ytest = device_df[y_col].iloc[train_idx:]

In [ ]:
scaler = StandardScaler()
scaler.fit(xtrain)

In [ ]:
# reg = LinearRegression().fit(scaler.transform(xtrain), ytrain)
# reg = GradientBoostingRegressor().fit(scaler.transform(xtrain), ytrain)
reg = RandomForestRegressor().fit(scaler.transform(xtrain), ytrain)
# reg = AdaBoostRegressor().fit(scaler.transform(xtrain), ytrain)
reg.score(scaler.transform(xtest), ytest)

In [ ]:
ytest.plot()
plt.plot(xtest.index, reg.predict(scaler.transform(xtest)))
plt.show()

# Cleanup

In [ ]:
# pbar.unregister()